In [322]:
import json
import pandas as pd
pd.set_option('display.max_rows', 500)

In [363]:
#open JSON file
with open("volunteer.json", "r") as read_file:
    data = json.load(read_file)

#make dictionary with ID as key, filter out null IDs
id_counts = {}
for i in data.keys():
    if i != "null":
        df = pd.DataFrame.from_records(data[i])
        counts = df['topic'].value_counts().to_dict()
        id_counts[i] = counts
        
#make data frame of User IDs and task counts
df1 = pd.DataFrame.from_dict(id_counts, orient='index').fillna(0)
df1 = df1.rename(columns={"Quoted Sources": "Quoted_sources"})

In [400]:
#Define function to return the IDs of users that have completed a specified amount of a specified task
def get_users(task, count):
    """
    Arguments:
        task {str}: The name of the task ("Evidence", "Quoted_sources", "Language", "Arguments", "Probability", "Reasoning")
        count {int}: minimum number of tasks completed
        
    Returns: 
        ids {list}: The list of user IDs that have completed the specified task and quantity

    """
    task = task.capitalize()
    if task not in ["Evidence", "Quoted_sources", "Language", "Arguments", "Probability", "Reasoning"]:
        raise ValueError('Task must be "Evidence", "Quoted_sources", "Language", "Arguments", "Probability", or "Reasoning"')
    else:
        temp = df1.query(task + ">= " + str(count), inplace = False)
        ids = temp.index.tolist()
        return ids

#Define function to return user IDs that have earned the specified badge in the specified task
def get_badges(task, level):
    """
    Arguments:
        level {str}: The name of the badge level ("badge", "bronze", "silver", "gold")
        
    Returns: 
        ids {list}: The list of user IDs that have earned the specified badge in the specified task

    """
    badges = {"Badge" : 50, "Bronze" : 100, "Silver" : 250, "Gold" : 500}
    level, task = level.capitalize(), task.capitalize()
    if level not in badges.keys():
        raise ValueError('Level must be "Badge", "Bronze", "Silver", or "Gold"')
    else:
        return get_users(task, badges[level])

#Define function to return all users that are owed badges
def get_all_badges():
    """
    Returns:
        total {dictionary}: A dictionary with key (Task name, Badge level) and value (list of IDs that are owed that badge)
    """
    total = {}
    badges = {"Badge" : 50, "Bronze" : 100, "Silver" : 250, "Gold" : 500} 
    tasks = ["Evidence", "Quoted_sources", "Language", "Arguments", "Probability", "Reasoning"]
    for task in tasks:
        for badge in badges:
            result = get_badges(task, badge)
            if result:
                total[task, badge] = result
    return total

In [402]:
total = get_all_badges()
total

{('Quoted_sources', 'Badge'): ['201'],
 ('Language', 'Badge'): ['196', '201', '205', '200']}